In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import os
import argparse
import numpy as np
from PIL import Image
from models import *
import torch
import torch.nn as nn
import torchvision.transforms as tfs
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import tqdm
import torch.nn.functional as F
import torchvision.transforms as T

In [4]:
totensor = T.ToTensor()
topil = T.ToPILImage()

def recover_image(image, init_image, mask, background=False):
    image = totensor(image)
    mask = totensor(mask)
    init_image = totensor(init_image)
    if background:
        result = mask * init_image + (1 - mask) * image
    else:
        result = mask * image + (1 - mask) * init_image
    return topil(result)

def preprocess(image):
    w, h = image.size
    w, h = map(lambda x: x - x % 32, (w, h))  # resize to integer multiple of 32
    image = image.resize((w, h), resample=Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.0 * image - 1.0

def prepare_mask_and_masked_image(image, mask):
    image = np.array(image.convert("RGB"))
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image).to(dtype=torch.float32) / 127.5 - 1.0

    mask = np.array(mask.convert("L"))
    mask = mask.astype(np.float32) / 255.0
    mask = mask[None, None]
    mask[mask < 0.5] = 0
    mask[mask >= 0.5] = 1
    mask = torch.from_numpy(mask)

    masked_image = image * (mask < 0.5)

    return mask, masked_image

def prepare_image(image):
    image = np.array(image.convert("RGB"))
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image).to(dtype=torch.float32) / 127.5 - 1.0

    return image[0]

def clamp(X, lower_limit, upper_limit):
    return torch.max(torch.min(X, upper_limit), lower_limit)

def tensorShow(tensors, titles=['haze']):
    fig = plt.figure()
    for tensor, tit, i in zip(tensors, titles, range(len(tensors))):
        img = make_grid(tensor)
        npimg = img.numpy()
        ax = fig.add_subplot(221+i)
        ax.imshow(np.transpose(npimg, (1, 2, 0)))
        ax.set_title(tit)
    plt.show()

In [ ]:


# abs=os.getcwd()+'/'
abs = os.getcwd()+'\\'





parser = argparse.ArgumentParser()
parser.add_argument('--task', type=str, default='its', help='its or ots')
parser.add_argument('--test_imgs', type=str,
                    default='test_imgs', help='Test imgs folder')
opt = parser.parse_args()


opt.task = "ots"
opt.test_imgs = "test_imgs"

dataset = opt.task
gps = 3
blocks = 19

# img_dir=abs+opt.test_imgs+'/'
img_dir = abs + opt.test_imgs+'\\'

# output_dir=abs+f'pred_FFA_{dataset}/'
output_dir = abs + f'pred_FFA_{dataset}\\'

print("pred_dir:", output_dir)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# model_dir=abs+f'trained_models/{dataset}_train_ffa_{gps}_{blocks}.pk'
model_dir = abs + f'trained_models\\{dataset}_train_ffa_{gps}_{blocks}.pk'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckp = torch.load(model_dir, map_location=device)
net = FFA(gps=gps, blocks=blocks)
net = nn.DataParallel(net)
net.load_state_dict(ckp['model'])
net.eval()


for im in os.listdir(img_dir):
    print(f'\r {im}', end='', flush=True)
    haze = Image.open(img_dir+im)
    haze1 = T.Compose([
        T.ToTensor(),
        T.Normalize(mean=[0.64, 0.6, 0.58], std=[0.14, 0.15, 0.152])
    ])(haze)[None, ::]
    haze_no = T.ToTensor()(haze)[None, ::]
    with torch.no_grad():
        pred = net(haze1)
    ts = torch.squeeze(pred.clamp(0, 1).cpu())
    tensorShow([haze_no, pred.clamp(0, 1).cpu()], ['haze', 'pred'])
    vutils.save_image(ts, output_dir+im.split('.')[0]+'_FFA.png')


In [5]:
# abs=os.getcwd()+'/'
abs = os.getcwd()+'\\'


dataset = "ots"
gps = 3
blocks = 19



# model_dir=abs+f'trained_models/{dataset}_train_ffa_{gps}_{blocks}.pk'
model_dir = abs + f'trained_models\\{dataset}_train_ffa_{gps}_{blocks}.pk'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckp = torch.load(model_dir, map_location=device)
net = FFA(gps=gps, blocks=blocks)
net = nn.DataParallel(net)
net.load_state_dict(ckp['model'])
net.eval()

DataParallel(
  (module): FFA(
    (g1): Group(
      (gp): Sequential(
        (0): Block(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (act1): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (calayer): CALayer(
            (avg_pool): AdaptiveAvgPool2d(output_size=1)
            (ca): Sequential(
              (0): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
              (1): ReLU(inplace=True)
              (2): Conv2d(8, 64, kernel_size=(1, 1), stride=(1, 1))
              (3): Sigmoid()
            )
          )
          (palayer): PALayer(
            (pa): Sequential(
              (0): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
              (1): ReLU(inplace=True)
              (2): Conv2d(8, 1, kernel_size=(1, 1), stride=(1, 1))
              (3): Sigmoid()
            )
          )
        )
        (1): Block(
          (conv1): Conv2d(64, 64, ker

In [6]:


cifar10_mean = (0.0, 0.0, 0.0)
cifar10_std = (1.0, 1.0, 1.0)
mu = torch.tensor(cifar10_mean).view(3,1,1).cuda()
std = torch.tensor(cifar10_std).view(3,1,1).cuda()

epsilon = 8
start_epsilon = 8
step_alpha = 2
seed = 160
num_img = 20
attack_iter = 50

upper_limit = ((1 - mu) / std)
lower_limit = ((0 - mu) / std)

epsilon = (epsilon / 255.) / std
start_epsilon = (start_epsilon / 255.) / std
step_alpha = (step_alpha / 255.) / std

input_image = Image.open("test_imgs/1400_2.png")
input_image = torch.unsqueeze(totensor(input_image).cuda(), 0)
delta = torch.zeros_like(input_image).cuda()
delta.requires_grad = True

clean_image = input_image

for k in range(attack_iter):
    k += 1
    dehazed_image = net(clean_image + delta)

    # wanna the dehazed output similar to the orginal, which means the model does nothing
    loss = F.mse_loss(clean_image.data, dehazed_image.float())
    loss.backward()
    grad = delta.grad.detach()
    d = delta
    d = clamp(d + step_alpha * torch.sign(grad), -epsilon, epsilon)
    delta.data = d
    delta.grad.zero_()

adv_image = clamp(clean_image + delta, lower_limit, upper_limit)
adv_image = torch.unsqueeze(adv_image.cuda(), 0)
adv_output = net(adv_image)

RuntimeError: CUDA out of memory. Tried to allocate 70.00 MiB (GPU 0; 6.00 GiB total capacity; 5.32 GiB already allocated; 0 bytes free; 5.34 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def pgd(X, model, eps=0.1, step_size=0.015, iters=40, clamp_min=0, clamp_max=1, mask=None):
    X_adv = X.clone().detach() + (torch.rand(*X.shape)*2*eps-eps).cuda()
    pbar = tqdm(range(iters))
    for i in pbar:
        actual_step_size = step_size - \
            (step_size - step_size / 100) / iters * i

        X_adv.requires_grad_(True)

        loss = (model(X_adv).latent_dist.mean).norm()

        pbar.set_description(
            f"[Running attack]: Loss {loss.item():.5f} | step size: {actual_step_size:.4}")

        grad, = torch.autograd.grad(loss, [X_adv])

        X_adv = X_adv - grad.detach().sign() * actual_step_size
        X_adv = torch.minimum(torch.maximum(X_adv, X - eps), X + eps)
        X_adv.data = torch.clamp(X_adv, min=clamp_min, max=clamp_max)
        X_adv.grad = None

        if mask is not None:
            X_adv.data *= mask

    return X_adv


In [ ]:
with torch.autocast('cuda'):
    X = preprocess(init_image).half().cuda()
    adv_X = pgd(X, 
                model=net, 
                clamp_min=-1, 
                clamp_max=1,
                eps=0.06, # The higher, the less imperceptible the attack is 
                step_size=0.02, # Set smaller than eps
                iters=100, # The higher, the stronger your attack will be
               )
    
    # convert pixels back to [0,1] range
    adv_X = (adv_X / 2 + 0.5).clamp(0, 1)

In [ ]:
# adv_image = recover_image(adv_image, init_image, mask_image, background=True)
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20,6))

ax[0].imshow(clean_image)
ax[1].imshow(adv_image)

ax[0].set_title('Source Image', fontsize=16)
ax[1].set_title('Adv Image', fontsize=16)

for i in range(4):
    ax[i].grid(False)
    ax[i].axis('off')
    
fig.tight_layout()
plt.show()